In [1]:
%load_ext autoreload
%autoreload 2
!sudo rm /tmp/libtpu_lockfile
import penzai
from penzai import pz
import jax.numpy as jnp
import jax_smi
import jax
jax_smi.initialise_tracking()
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
from micrlhf.llama import LlamaTransformer
from micrlhf.scan import sequential_to_scan
from micrlhf.flash import flashify

import gc
llama = LlamaTransformer.from_pretrained(("models/llama-3-70b-1.gguf", "models/llama-3-70b-2.gguf"),
                                         device_map="auto:mp=4", load_on_cpu=True)
llama = sequential_to_scan(llama)
# llama = flashify(llama)
llama = llama.to_tpu()

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-70B")
prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
] * 1_000, tokenize=False)

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
from penzai.toolshed import jit_wrapper
from micrlhf.flash import flashify
tokens = pz.nx.wrap([(tokenizer.encode(prompt))[:64]] * 256, "batch", "seq")
inputs = llama.inputs.from_basic_segments(tokens)
# llama_jitted = jit_wrapper.Jitted(flashify(llama))
llama_jitted = jit_wrapper.Jitted(llama)

In [13]:
@jax.jit
def lfn(llama_jitted, inputs):
    logits = llama_jitted(inputs)
    loss = -pz.nx.nmap(lambda l, t: jnp.take_along_axis(jax.nn.log_softmax(l[:-1], -1), t[1:, None], 1).mean())(logits.untag("seq", "vocabulary"), tokens.untag("seq"))
    return loss
print(lfn(llama_jitted, inputs))

<NamedArray bfloat16(| batch:256) ≈3.0 ±0.0075 [≥3.0, ≤3.0] nonzero:256 (wrapping jax.Array)>


In [6]:
1/0
from tqdm.auto import trange
import dataclasses
for i in (bar := trange(16, 256)):
    logits = llama_jitted(inputs)
    new_tokens = logits[{"seq": i}].untag("vocabulary").argmax()
    tokens = pz.nx.nmap(lambda a, b: a.at[i+1].set(b))(tokens.untag("seq"), new_tokens).tag("seq")
    bar.set_postfix(txt=tokenizer.decode(tokens[{"batch": 0}].unwrap("seq")))
    inputs = dataclasses.replace(inputs, tokens=tokens)

ZeroDivisionError: division by zero